## Check simple baselines (taking only a single row from the dynamic data, taking the most prominent one (q25)) and do a quick evaluation of the quality of the mapping

In [57]:
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
# load utils
from src.category_descriptor_selection import *
from src.make_dataset.split_data import *
from src.make_dataset import split_data

In [37]:
FEATURES_PATH = Path("../data/raw/features")
OUTPUT_PATH = Path("../data/processed")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
HIERARCHY_PATH = Path("../data/hierarchy_map.csv")

In [38]:
features_3900 = pd.read_parquet(OUTPUT_PATH / "features_3900.parquet")
hierarchy = pd.read_csv(HIERARCHY_PATH)

In [39]:
cv5_splits = load_kfold_splits()

### Baselines - q25, single row

In [92]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

def evaluate_baseline(X_baseline, y_valence, y_arousal, tr_ids, va_ids, name="Baseline"):
    """Evaluate a baseline feature set on a single train/val split"""
    
    # Get train/val sets
    tr_idx = X_baseline.index.intersection(tr_ids)
    va_idx = X_baseline.index.intersection(va_ids)
    
    # Impute with train medians
    med = X_baseline.loc[tr_idx].median(numeric_only=True)
    Xtr = X_baseline.loc[tr_idx].fillna(med)
    Xva = X_baseline.loc[va_idx].fillna(med)
    
    # Train and evaluate valence
    rf_v = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
    rf_v.fit(Xtr, y_valence.loc[tr_idx])
    pred_v = rf_v.predict(Xva)
    rmse_v = np.sqrt(((y_valence.loc[va_idx] - pred_v)**2).mean())
    r2_v = r2_score(y_valence.loc[va_idx], pred_v)
    
    # Train and evaluate arousal
    rf_a = RandomForestRegressor(n_estimators=200, random_state=43, n_jobs=-1)
    rf_a.fit(Xtr, y_arousal.loc[tr_idx])
    pred_a = rf_a.predict(Xva)
    rmse_a = np.sqrt(((y_arousal.loc[va_idx] - pred_a)**2).mean())
    r2_a = r2_score(y_arousal.loc[va_idx], pred_a)
    
    # Joint metrics
    joint_rmse = 0.5 * (rmse_v + rmse_a)
    joint_r2 = 0.5 * (r2_v + r2_a)
    
    print(f"\n{name} Results:")
    print(f"  Features: {X_baseline.shape[1]}")
    print(f"  Valence RMSE: {rmse_v:.4f}, R²: {r2_v:.4f}")
    print(f"  Arousal RMSE: {rmse_a:.4f}, R²: {r2_a:.4f}")
    print(f"  Joint RMSE:   {joint_rmse:.4f}, R²: {joint_r2:.4f}")
    
    return {
        'valence_rmse': rmse_v, 'valence_r2': r2_v,
        'arousal_rmse': rmse_a, 'arousal_r2': r2_a,
        'joint_rmse': joint_rmse, 'joint_r2': joint_r2
    }

In [93]:
# filter columns ending with _q25 (descriptor that carries on average the strongest signal)
X_q25 = features_3900[[c for c in features_3900.columns if c.endswith('_q25')]]
print(f"Q25 baseline: {X_q25_only.shape[1]} features")

Q25 baseline: 260 features


In [96]:
tr, va, te = load_splits_triplet()

In [97]:
esults_q25 = evaluate_baseline_single_split(X_q25, y_valence_mean, y_arousal_mean, tr, va, "Q25 Only")


Q25 Only Results:
  Features: 260
  Valence RMSE: 0.8301, R²: 0.5010
  Arousal RMSE: 0.8293, R²: 0.6099
  Joint RMSE:   0.8297, R²: 0.5555


In [98]:
# Baseline 2: Mean only (the average signal from the 45 seconds)
X_mean = features_3900[[c for c in features_3900.columns if c.endswith('_mean')]]
results_mean = evaluate_baseline(X_mean, y_valence_mean, y_arousal_mean, tr, va, "Mean Only")


Mean Only Results:
  Features: 260
  Valence RMSE: 0.8298, R²: 0.5014
  Arousal RMSE: 0.8436, R²: 0.5964
  Joint RMSE:   0.8367, R²: 0.5489


In [99]:
def create_features_first_row_only(features_path: Path) -> pd.DataFrame:
    """Create features using only the first frame of each song"""
    from src.io import load_opensmile_csv
    from src.cleaning import clean
    rows = []
    ids = []
    
    csv_files = sorted(features_path.glob("*.csv"))
    
    for csv_path in tqdm(csv_files, desc="Getting first frame only"):
        sid = int(csv_path.stem)
        # Load and clean
        df = load_opensmile_csv(csv_path, sep=';')
        df_clean = clean(df)
            
        # Quality check
        if df_clean.shape[0] < 1 or df_clean.shape[1] < 10:
            continue
            
        # Just take first row, convert to Series
        first_row = df_clean.iloc[0]
            
        rows.append(first_row)
        ids.append(sid)
    
    # Create DataFrame from first rows
    first_frame_df = pd.DataFrame(rows, index=ids)
    first_frame_df.index.name = 'song_id'
    first_frame_df.index = first_frame_df.index.astype(int)
    first_frame_df = first_frame_df.sort_index()
    
    print(f"Created first-frame dataset: {first_frame_df.shape}")
    
    return first_frame_df

In [100]:
# Baseline 3: first row
X_first_row = create_features_first_row_only(FEATURES_PATH)

Getting first frame only: 100%|█████████████████████████████████████████████████████| 1802/1802 [00:59<00:00, 30.30it/s]

Created first-frame dataset: (1802, 260)


In [101]:
results_first_row = evaluate_baseline(X_first_row, y_valence_mean, y_arousal_mean, tr, va, "First row Only")


First row Only Results:
  Features: 260
  Valence RMSE: 1.0562, R²: 0.1922
  Arousal RMSE: 1.0519, R²: 0.3724
  Joint RMSE:   1.0541, R²: 0.2823


### Recap:
- [Q25] over Rows : Joint RMSE:   0.8297, R²: 0.5555
- [Mean] over Rows: Joint RMSE:   0.8367, R²: 0.5489
- [Single] Row only: Joint RMSE:   1.0541, R²: 0.2823

### Evaluate Mapping quality

In [40]:
# Load labels
labels = pd.read_parquet("../data/processed/core_dataset.parquet")
labels = labels.set_index("song_id")
labels.index = labels.index.astype(int)

# Align features and labels
common_ids = features_3900.index.intersection(labels.index)

X = features_3900.loc[common_ids] # apply the alignment
y_valence_mean = labels.loc[common_ids, "valence_mean"]
y_arousal_mean = labels.loc[common_ids, "arousal_mean"]

In [29]:
def mapping_coverage(hierarchy_map, agg_df):
    bases_in_X = sorted({base_of(c) for c in agg_df.columns if '_PC' not in c})
    mapped = set(hierarchy_map['feature'])
    cov = len(set(bases_in_X) & mapped) / max(1, len(bases_in_X))
    print(f"Coverage on current design: {cov:.1%} ({len(set(bases_in_X)&mapped)}/{len(bases_in_X)})")

    amb = (hierarchy_map.groupby('feature')['perceptual'].nunique())
    amb = amb[amb > 1]
    print(f"Ambiguous features (mapped to >1 perceptual): {len(amb)}")
    if len(amb): display(amb.head(15))

mapping_coverage(hierarchy, features_3900)

Coverage on current design: 100.0% (260/260)
Ambiguous features (mapped to >1 perceptual): 0


In [16]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor

def group_ablation_cv(X, y, hierarchy_map, splits, n_estimators=300):
    if any('_PC' in c for c in X.columns):
        raise ValueError("Run on non-PCA (LLD) designs.")

    def cv_rmse(X_):
        rmses=[]
        for tr_ids, va_ids in splits:
            tr = X_.index.intersection(tr_ids); va = X_.index.intersection(va_ids)
            med = X_.loc[tr].median()
            Xtr, Xva = X_.loc[tr].fillna(med), X_.loc[va].fillna(med)
            ytr, yva = y.loc[tr], y.loc[va]
            m = RandomForestRegressor(n_estimators=n_estimators, random_state=42, n_jobs=-1)
            m.fit(Xtr, ytr); p = m.predict(Xva)
            rmses.append(float(np.sqrt(((yva - p)**2).mean())))
        return float(np.mean(rmses))

    base2per = hierarchy_map.set_index('feature')['perceptual'].to_dict()
    per2cols = {}
    for c in X.columns:
        g = base2per.get(base_of(c))
        if g: per2cols.setdefault(g, []).append(c)

    rmse_full = cv_rmse(X)
    rows=[{'perceptual':'__FULL__','rmse':rmse_full,'delta':0.0,'dropped':0}]
    for g, cols in per2cols.items():
        rmse = cv_rmse(X.drop(columns=cols, errors='ignore'))
        rows.append({'perceptual':g, 'rmse':rmse, 'delta':rmse - rmse_full, 'dropped':len(cols)})
    return pd.DataFrame(rows).sort_values('rmse')

In [21]:
ab_tbl = group_ablation_cv(features_3900, y_valence_mean, hierarchy, cv5_splits)

In [22]:
print(ab_tbl)

            perceptual      rmse     delta  dropped
4           dissonance  0.830694 -0.003224      480
0             __FULL__  0.833918  0.000000        0
2      tonal_stability  0.834852  0.000934      960
6            minorness  0.835042  0.001124      240
1        melodiousness  0.835718  0.001799      600
3         articulation  0.836310  0.002392      660
7  rhythmic_complexity  0.838138  0.004220      600
5   rhythmic_stability  0.838147  0.004228      360


### checking the new map

In [42]:
hierarchy = pd.read_csv(HIERARCHY_PATH)

In [44]:
mapping_coverage(hierarchy, features_3900)

Coverage on current design: 100.0% (260/260)
Ambiguous features (mapped to >1 perceptual): 0


In [53]:
hierarchy.head(5)

,feature,perceptual,musical,acoustic
0,F0final_sma_amean,melodiousness,pitch,fundamental_freq
1,F0final_sma_de_amean,melodiousness,pitch,fundamental_freq
2,F0final_sma_de_stddev,tonal_stability,pitch,pitch_delta
3,F0final_sma_stddev,tonal_stability,pitch,pitch_delta
4,audSpec_Rfilt_sma[0]_amean,unmapped,other,unknown


In [54]:
ab_tbl = group_ablation_cv(features_3900, y_valence_mean, hierarchy, cv5_splits)

In [55]:
print(ab_tbl)

            perceptual      rmse     delta  dropped
5           dissonance  0.829552 -0.004366      240
6  rhythmic_complexity  0.832560 -0.001358      450
2        melodiousness  0.833146 -0.000773       60
0             __FULL__  0.833918  0.000000        0
1      tonal_stability  0.834195  0.000277      120
7   rhythmic_stability  0.834645  0.000727       30
3         articulation  0.836566  0.002648      720
4             unmapped  0.852741  0.018823     2280


Here, dropping 158 unmapped features corresponding to 2280 base features * their descriptors, leads to only a 2% decrease in performance, validated across 5 folds, which means that the cost of dropping more features is minimal for interpretability (better mapping quality) and feature reduction

### checking the old mapping quality

In [30]:
assert hierarchy['feature'].is_unique
assert not hierarchy[['perceptual','musical','acoustic']].isna().any().any()

# human spot-check (3 exemplars per perceptual group)
spot = (hierarchy.groupby('perceptual', group_keys=False)
                     .apply(lambda g: g.sample(min(3, len(g)), random_state=0)))
print(spot.to_string(index=False))

                                    feature          perceptual  musical            acoustic
                  jitterLocal_sma_de_stddev        articulation    voice              jitter
                       pcm_zcr_sma_de_amean        articulation   rhythm       zero_crossing
                       jitterDDP_sma_stddev        articulation    voice              jitter
               audSpec_Rfilt_sma[25]_stddev          dissonance spectral        audspec_high
              pcm_fftMag_mfcc_sma[13]_amean          dissonance   timbre           mfcc_high
                audSpec_Rfilt_sma[25]_amean          dissonance spectral        audspec_high
             audSpec_Rfilt_sma_de[13]_amean       melodiousness temporal   audspec_delta_mid
             audSpec_Rfilt_sma_de[12]_amean       melodiousness temporal   audspec_delta_mid
            audSpec_Rfilt_sma_de[14]_stddev       melodiousness temporal   audspec_delta_mid
     pcm_fftMag_fband1000-4000_sma_de_amean           minorness   timb

/tmp/ipykernel_8825/298277679.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(min(3, len(g)), random_state=0)))


Assessment on the projects concerned mapping (musical and acoustic are just for quick analysis, perceptual is the one used throughout the project for linking the emotion labels into human-relevant categories that influence the emotion.
Missed-messy mappings:
#### Articulation(0/3):
- jitterLocal_sma_de_stddev -> articulation | accurate mapping that can be found -> phonation/voice quality, weak link for articulation
- pcm_zcr_sma_de_amean -> articulation | accurate mapping that can be found -> noisiness/percussiveness/brightness, weak link for articulation
- jitterDDP_sma_stddev -> articulation | accurate mapping that can be found -> phonation stability, weak link for articulation
#### Minorness(0/3).
- pcm_fftMag_fband1000–4000_sma_de_amean -> minorness |  weak/highly speculative link for minorness
- pcm_fftMag_fband250–650_sma_de_stddev -> minorness |  weak/highly speculative link for
- pcm_fftMag_mfcc_sma[5]_amean-> minorness |  weak/highly speculative link for
#### Dissonance(2/3)
- pcm_fftMag_mfcc_sma[13]_amean -> dissonance | accurate mapping that can be found -> spectral envelope/timbre, weak link for dissonance
#### Rhythmic_complexity(2/3)
- pcm_fftMag_spectralEntropy_sma_de_amean -> rhythmic_complexity | accurate mapping that can be found -> tonality/noise-likeness, weak link for rhythmic complexity
#### Tonal_Stability(1/3)
- pcm_fftMag_spectralRollOff25.0_sma_de_amean -> tonal_stability | accurate mapping that can be found -> energy percentile/brightness, weak link for tonal stability
- pcm_fftMag_mfcc_sma_de[6]_stddev -> tonal_stability | accurate mapping that can be found -> timbre , weak link for tonal stability
#### Overall:
High-band filtered spectrum -> dissonance rows plausible; increased high-freq energy/variance can relate to harshness/roughness, so they’re at least defensible without stronger evidence.

Rhythmic_complexity/stability assignments for low-band (index ~5) energy/deltas also arguable.
#### Overall the mapping should not be considered accurate and needs refinement.
- Improvement idea: For the 7 perceptual categories, use a DL model that takes mel spectrograms as inputs and learns to predict them in a supervised manner, using the Mid-level perceptual musical features dataset (5000 songs annotated for 7 categories by musicians), then adapt it to the current dataset

### new map

In [56]:
assert hierarchy['feature'].is_unique
assert not hierarchy[['perceptual','musical','acoustic']].isna().any().any()

# human spot-check (3 exemplars per perceptual group)
spot = (hierarchy.groupby('perceptual', group_keys=False)
                     .apply(lambda g: g.sample(min(3, len(g)), random_state=0)))
print(spot.to_string(index=False))

                                 feature          perceptual       musical             acoustic
                  jitterDDP_sma_de_amean        articulation voice_quality               jitter
          audSpec_Rfilt_sma_de[12]_amean        articulation        rhythm    audspec_delta_mid
           audSpec_Rfilt_sma_de[9]_amean        articulation        rhythm audspec_delta_lowmid
                       logHNR_sma_stddev          dissonance voice_quality          noise_ratio
   pcm_fftMag_psySharpness_sma_de_stddev          dissonance        timbre            sharpness
   pcm_fftMag_spectralCentroid_sma_amean          dissonance        timbre    spectral_centroid
         voicingFinalUnclipped_sma_amean       melodiousness         voice              voicing
      voicingFinalUnclipped_sma_de_amean       melodiousness         voice              voicing
                    F0final_sma_de_amean       melodiousness         pitch     fundamental_freq
                         minorness_score

/tmp/ipykernel_8825/298277679.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(min(3, len(g)), random_state=0)))


Some mappings still seem highly speculative.The current rule-based approach lacks the sophistication to capture perceptual-acoustic relationships accurately.
#### Future work:
##### Implement a supervised deep learning approach:

- Training: Use the Mid-level Perceptual Musical Features dataset (5000 expert-annotated songs)
- Architecture: CNN/Transformer on mel-spectrograms for direct perceptual prediction
- Adaptation: Transfer learning to current dataset maintaining learned perceptual representations
- Validation: Cross-reference with current mappings to retain high-confidence associations

This would replace brittle rules with learned representations that better capture the complex, non-linear relationships between acoustic features and perceptual dimensions.